In [1]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australianBushfireDB
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires

In [3]:
byStateurl = 'https://en.wikipedia.org/wiki/List_of_fires_and_impacts_of_the_2019-20_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'

In [4]:
historicalresponse = requests.get(historicalurl)

In [5]:
soup = BeautifulSoup(historicalresponse.text, 'html.parser')

In [6]:
table = soup.find('table', class_="wikitable")
# header_row = soup.find('tr')
table_headers = soup.find_all("tr")[0:2]
# table_headers = table_head.find_all('th', class_="headerSort")


In [7]:
# table_headers = table_head.find('tr')
headers = []
for table_header in table_headers:
    try:
        value = table_header.text
#         value = value.split("\n")
        headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(headers)
print(len(headers))

['\nDate\n\nName or description\n\nState(s) /territories\n\nArea burned (approx.)\n\nFatalities\n\nProperties damaged\n\nNotes\n', '\nha\n\nacres\n\nHomes(destroyed)\n\nOther buildings\n\nOther damage\n']
2


In [8]:
table = soup.find('table', class_="wikitable")
table_body = table.find("tbody")
table_row = table_body.find_all('tr')[2:]

In [35]:

data = []
for row in table_row:
#     print(len(row))
    datarow = []
    table_data = row.find_all('td')
    for tdata in table_data:
#         try:
        value = tdata.text
#             value.split("\n\n")
#             value = value.replace("\n", "")
        datarow.append(value)

#         except AttributeError as e:
#             print(e)
            
    data.append(datarow)
        
print(data)

[['6 February 1851\n', 'Black Thursday bushfires\n', 'Victoria\n', '5,000,000\n', '12,000,000\n', 'approx. 12\n', 'Nil\n', 'Nil\n', '1 million sheepthousands of cattle\n', '[7][8]\n'], ['1 February 1898\n', 'Red Tuesday bushfires\n', 'Victoria\n', '260,000\n', '640,000\n', '12\n', 'Nil\n', '2,000\n', '\n', '[8][9][10]\n'], ['February – March 1926\n', '1926 bushfires\n', 'Victoria\n', '390,000\n', '960,000\n', '60\n', '1,000\n', 'Nil\n', '\n', '[11]\n'], ['December 1938 – January 1939\n', 'Black Friday bushfires\n', 'Victoria\n', '2,000,000\n', '4,900,000\n', '71\n', '3,700\n', 'Nil\n', '\n', '[5]\n'], ['14 January – 14 February 1944\n', '1944 Victorian bushfires\n', 'Victoria\n', '1,000,000\n', '2,500,000\n', '15–20\n', 'approx. 500\n', 'Nil\n', '\n', '[8]\n'], ['18 November 1944\n', '1944 Blue Mountains bushfire\n', 'New South Wales\n', '\n', '\n', 'Nil\n', 'approx. 40\n', 'Nil\n', '\n', '[12][13]\n'], ['November 1951 – January 1952\n', '1951–52 bushfires\n', 'Victoria\n', '4,000,000\

In [36]:
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8,9
0,6 February 1851\n,Black Thursday bushfires\n,Victoria\n,"5,000,000\n","12,000,000\n",approx. 12\n,Nil\n,Nil\n,1 million sheepthousands of cattle\n,[7][8]\n
1,1 February 1898\n,Red Tuesday bushfires\n,Victoria\n,"260,000\n","640,000\n",12\n,Nil\n,"2,000\n",\n,[8][9][10]\n
2,February – March 1926\n,1926 bushfires\n,Victoria\n,"390,000\n","960,000\n",60\n,"1,000\n",Nil\n,\n,[11]\n
3,December 1938 – January 1939\n,Black Friday bushfires\n,Victoria\n,"2,000,000\n","4,900,000\n",71\n,"3,700\n",Nil\n,\n,[5]\n
4,14 January – 14 February 1944\n,1944 Victorian bushfires\n,Victoria\n,"1,000,000\n","2,500,000\n",15–20\n,approx. 500\n,Nil\n,\n,[8]\n
...,...,...,...,...,...,...,...,...,...,...
63,25 November – 2 December 2015\n,2015 Pinery bushfire\n,South Australia\n,"85,000\n","210,000\n",2\n,91\n,Nil\n,\n,[66][67]\n
64,January 2016\n,2016 Murray Road bushfire (Waroona and Harvey)\n,Western Australia\n,"69,165\n","170,910\n",2\n,181\n,Yarloop Workshops\n,Thousands of hectares of Lane Poole ReservePro...,[68]\n
65,11 – 14 February 2017\n,2017 New South Wales bushfires\n,New South Wales\n,"52,000\n","130,000\n",Nil\n,35\n,Nil\n,\n,[69]\n
66,early February 2019\n,Tingha bushfire\n,New South Wales\n,"23,419\n","57,870\n",Nil\n,19\n,57\n,LivestockVineyards\n,[70][71]\n


In [ ]:
tables = pd.read_html(historicalurl)
tables